<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/Advance_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-05-27 14:25:44--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1748359545&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODM1OTU0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [2]:
 pip install langchain-community langchain-core langchain llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127048 sha256=e8b3c2d98fbf6a40fad14639485a028592c9d9ebc1cc34e6518cf9164971bca3
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [3]:
from langchain import LlamaCpp
llm = LlamaCpp(
  model_path="Phi-3-mini-4k-instruct-fp16.gguf",
  n_gpu_layers=-1,
  max_tokens=500,
  n_ctx=2048,
  seed=42,
  verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [4]:
llm.invoke("Hi! My name is Maarten. What is 1+1?")

''

In [9]:
from langchain import PromptTemplate
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(template=template, input_variables=["input_prompt"])

basic_chain = prompt | llm

basic_chain.invoke({
  'input_prompt':'Hi! My name is Maarten. What is 1 + 1?'
})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

In [12]:
from langchain import LLMChain
template = """<s><|user|>
Create a titlte for a story about {summary}. only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(prompt=title_prompt, llm=llm, output_key="title")

title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Loss: A Tale of a Motherless Journey"'}

In [15]:
template="""<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use oly two sentences.<|end|>
<|assistant|>"""

character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"])
character = LLMChain(prompt=character_prompt, llm=llm, output_key="character")

In [16]:
template = """<s><|user|>
Create a stroy about {summary} with the {title}. The main character is:{character}.
Only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>"""

story_prompt = PromptTemplate(template=template, input_variables=["summary","title", "character"])
story = LLMChain(prompt=story_prompt, llm=llm, output_key="story")

In [17]:
llm_chain = title | character | story

In [18]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of a Mother\'s Love: A Girl\'s Journey Through Loss and Healing"',
 'character': ' The protagonist, Emily, is a resilient yet vulnerable young girl grappling with the profound grief of losing her mother. Despite her sorrow, she embarks on an emotional journey to find solace in cherished memories and discover inner strength through love and healing.',
 'story': " Emily's heart ached as the whispers of her mother's love echoed through the empty rooms, reminding her that she was no longer there. Grief weighed heavy on young shoulders but propelled Emily forward to embark on a journey—a quest for solace and healing amidst sorrow. With each tender memory etched in her heart, and every step taken with unwavering determination, the resilient girl discovered that love could mend even the deepest wounds. Through sharing stories of laughter and tears, finding strength in small moments shared with family and friends, Emily embraced 